In [22]:
# setup ODE
data_path = "../data/"
traces_path = "5_ode.csv"

import pandas as pd
df = pd.read_csv(data_path+traces_path)
# print(df.head(5))

# collect dialogues
case = 0
dialogues = []
for index, utterance in df.iterrows():
    if utterance['case ID'] != case:
        dialogues.append('')
        case = utterance['case ID']
    elif utterance['New'] != 'Hello':
        dialogues[-1] += utterance['New'][0]
print(len(dialogues))
print(dialogues)

26
['QRQRQRQRQAF', 'QRFAQRQRQAF', 'QRQRQAF', 'AQRQRQRQAF', 'RQRQRQAQRQRF', 'RQRQAFRFRQRQAQRQRFRFRQAFAFA', 'QRQAQRQRFRF', 'QRQRQAQAFRFAQRFRF', 'RQAQRQAFRQAQRQA', 'QRQRQRQRQRFRQRFRFRFRQRQRQRF', 'RFRFRQRFRQAFRFRQRFRFRQRQRQRQRQRQRQA', 'QAFRF', 'QRQRQRQRQRFRFRFRFRQRQRFAQRFRF', 'QRQAQAFAFAFRFAFA', 'RQRFRQRFRFA', 'QRQRQAFRF', 'AQAAF', 'QRFRFRFRQRFRFRFRQRFRQAFRQRQAFA', 'QRQAQRFAFA', 'QRQRQAQAF', 'RQAQAF', 'RQAQRQAFRFRFRFAF', 'QRQRQRFRQAF', 'RQRQAFRQAQAFAQRFRQRFAFAFA', 'QAQAQAFRFAF', 'RFAFA']


In [26]:
# find loops

import string
chars = string.ascii_lowercase

def mine_simple_loops(dialogues, loops_alphabet={}): 
    # mine loops (linear), e.g. QRQ, i.e. s[0] == s[2] -> s[0:1] is a loop
    loops = []
    s = ''
    for d in 'o'.join(dialogues):
        if d == 'o':
            # reset pattern
            s = ''
            continue
        s += d
        if len(s) == 3:
            # check if loop
            if s[0] == s[2]:
                loop = s
                if loop not in loops:
                    loops.append(loop)
            # slide over
            s = s[1:]
    if not loops:
        return None, loops_alphabet
    print(loops)

    # group loops (linear)
    for loop in loops:
        # normalise loop label
        loop_label = ''.join(list(set(loop)))
        if loop_label not in loops_alphabet:
            loops_alphabet[loop_label] = chars[len(loops_alphabet)]
    print(loops_alphabet)
    
    # transform log by replacing all loops with loop ids
    trace = ''
    s = ''
    for d in 'o'.join(dialogues):
        s += d
        if s in loops:
            loop_label = ''.join(list(set(s)))
            loop_id = str(loops_alphabet[loop_label])
            # do not repeat loop symbols
            if trace == '' or trace[-1] != loop_id:
                trace += loop_id
            s = ''
        elif len(s) == 3:
            trace += s[0]
            # slide over
            s = s[1:]
    trace += s
    return trace.split('o'), loops_alphabet


def build_hpm(dialogues):
    print(dialogues)
    dialogues, loops_alphabet = mine_simple_loops(dialogues)
    return dialogues, loops_alphabet


dialogues_2, loops_alphabet = build_hpm(dialogues)
print(dialogues_2)
# print(loops_alphabet)

['QRQRQRQRQAF', 'QRFAQRQRQAF', 'QRQRQAF', 'AQRQRQRQAF', 'RQRQRQAQRQRF', 'RQRQAFRFRQRQAQRQRFRFRQAFAFA', 'QRQAQRQRFRF', 'QRQRQAQAFRFAQRFRF', 'RQAQRQAFRQAQRQA', 'QRQRQRQRQRFRQRFRFRFRQRQRQRF', 'RFRFRQRFRQAFRFRQRFRFRQRQRQRQRQRQRQA', 'QAFRF', 'QRQRQRQRQRFRFRFRFRQRQRFAQRFRF', 'QRQAQAFAFAFRFAFA', 'RQRFRQRFRFA', 'QRQRQAFRF', 'AQAAF', 'QRFRFRFRQRFRFRFRQRFRQAFRQRQAFA', 'QRQAQRFAFA', 'QRQRQAQAF', 'RQAQAF', 'RQAQRQAFRFRFRFAF', 'QRQRQRFRQAF', 'RQRQAFRQAQAFAQRFRQRFAFAFA', 'QAQAQAFRFAF', 'RFAFA']
['QRQ', 'RQR', 'QAQ', 'FRF', 'RFR', 'AFA', 'FAF', 'AQA']
{'QR': 'a', 'AQ': 'b', 'FR': 'c', 'AF': 'd'}
['aAF', 'QRFAaRQAF', 'aRQAF', 'AaQAF', 'aAaRF', 'aQAcabacRQdFA', 'aAacF', 'aRbAcAQcF', 'RbRQAFRbRQA', 'acQcaRF', 'cFaFRQAcacaRQA', 'QAc', 'acaRFAQcF', 'abdAcd', 'aFacA', 'aRQAc', 'bAF', 'QcacQcQAFaQd', 'aAQRdA', 'aRbAF', 'RbAF', 'RbRQAcd', 'aFRQAF', 'aQAFRbdQcQRd', 'bcAF', 'RdA']


In [85]:
# find how the loops are connected: separate model for each loop
from collections import defaultdict, Counter

trace = ''.join(['o' + d for d in dialogues_2])

# most frequent patterns
enter_loop = defaultdict(Counter)
exit_loop = defaultdict(Counter)

for i in range(len(trace)):
    if trace[i] in loops_alphabet.values():
        # how did we enter this loop
        if trace[i-1] == 'o' or trace[i-1] not in chars:
            enter_loop[trace[i]][trace[i-1]] += 1
        # how did we exit this loop
        if i+2 < len(trace) and trace[i+2] not in chars:
            exit_loop[trace[i]][trace[i+2]] += 1

for loop, loop_id in loops_alphabet.items():
#     print(loop)
    component = ''
    most_common_start = enter_loop[loop_id].most_common(1)[0][0]
#     if most_common_start != 'o':
    component += most_common_start
    component += loop
#     print(exit_loop[loop_id].most_common())
    for l in exit_loop[loop_id].most_common():
        most_common_end = l[0]
        if most_common_end in loop or most_common_end in chars:
            continue
        component += most_common_end
        break
    print(component)


oQRF
RAQF
AFRQ
RAF


In [98]:
# find how the loops are connected: separate model for each loop
from collections import defaultdict, Counter
print(dialogues_2)
trace = 'o'.join(dialogues_2)

tail = ''
starts = defaultdict(list)
ends = defaultdict(list)

for d in trace:
    if d in loops_alphabet.values():
        loop = d
        if tail <= 2:
            starts[loop].append(tail)
        tail = ''
    elif d == 'o' and loop:
        if tail <= 2:
            ends[loop].append(tail)
        tail = ''
        loop = None
    else:
     tail += d

# print(starts)
# print(ends)

for loop, loop_id in loops_alphabet.items():
    



SyntaxError: unexpected EOF while parsing (<ipython-input-98-be056b654d82>, line 29)